<a href="https://colab.research.google.com/github/Beerschtein/PyTorch/blob/main/HW_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание_9

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.2 MB/s 
     |████████████████████████████████| 120 kB 61.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 


In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 7.6 MB/s 


### Загрузим библиотеки

In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from transformers import pipeline
from tqdm import tqdm
from collections import Counter
from torchmetrics import F1Score
from transformers import BertTokenizer, BertForSequenceClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [10]:
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

In [12]:
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[    2, 33278, 22723,   871, 24302,  3464, 10880,     3,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [13]:
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD] [PAD]


### Скачаем и подготовим данные

In [14]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2022-09-14 19:10:16--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 74.125.199.138, 74.125.199.113, 74.125.199.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.199.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pl81i80691kd78keok3ut2rqkturo3o5/1663182600000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=9848defc-bd88-4a29-83a6-22486817c217 [following]
--2022-09-14 19:10:21--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pl81i80691kd78keok3ut2rqkturo3o5/1663182600000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=9848defc-bd88-4a29-83a6-22486817c217
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 74.125.197.132, 2607:

In [15]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [16]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [17]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [18]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [25]:
idx = 46
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

после рецензии   , от похода на аватар невозможно отказаться :) купили билеты в золотой вавилон на 20:00 еще кто нибудь и
label is 1


[{'label': 'LABEL_1', 'score': 0.5242937803268433}]

In [20]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [21]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')
               #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [22]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [26]:
for txt, lbl in train_loader:
    print(txt.keys()) 
    print(txt['input_ids'].shape) 
    print(txt['attention_mask'].shape) 
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])
torch.Size([64, 1, 10])


In [27]:
class BertForSequenceClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.pretrained_model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
        self.dropout = nn.Dropout(dropout)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.pretrained_model(input_ids=x, attention_mask=mask, return_dict=False)[0]  #(B, 2)
        dropout_output = self.dropout(pooled_output)
        out = self.sigm(dropout_output)
        return out

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [29]:
model = BertForSequenceClassifier().to(device)
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.pretrained_model.classifier.parameters()]))

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassifier(
  (pretrained_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(83828, 312, padding_idx=0)
        (position_embeddings): Embedding(2048, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312, ou

### Посмотрим метрику валидационного датасета на модели без обучения

In [30]:
f1_val = F1Score().to(device)
model.eval()

total_acc_val = 0

for val_input, val_label in valid_loader:

    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    f1_val(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {f1_val.compute().item():.3f}')

Val accuracy: 0.561, F1 score: 0.561


### Зададим критерий и оптимизатор

In [32]:
criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.pretrained_model.parameters(), lr=0.0001)
# optimizer = Adam(model.pretrained_model.parameters(), lr=0.001)

### Дообучим модель и посчитаем метрики

In [33]:
f1_train = F1Score().to(device)
f1_valid = F1Score().to(device)

epochs = 3

for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)
        
        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        f1_train(output, train_label)

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

        f1_valid(output, val_label)
     
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Train f1: {f1_train.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f} \
        | Val f1: {f1_valid.compute().item(): .3f}')
    
    f1_train.reset()
    f1_valid.reset()

100%|██████████| 2836/2836 [01:05<00:00, 43.39it/s]


Epochs: 1 | Train Loss:  0.010         | Train Accuracy:  0.620         | Train f1:  0.620         | Val Loss:  0.010         | Val Accuracy:  0.665         | Val f1:  0.665


100%|██████████| 2836/2836 [01:05<00:00, 43.42it/s]


Epochs: 2 | Train Loss:  0.010         | Train Accuracy:  0.647         | Train f1:  0.647         | Val Loss:  0.010         | Val Accuracy:  0.679         | Val f1:  0.679


100%|██████████| 2836/2836 [01:05<00:00, 43.20it/s]


Epochs: 3 | Train Loss:  0.010         | Train Accuracy:  0.675         | Train f1:  0.675         | Val Loss:  0.010         | Val Accuracy:  0.677         | Val f1:  0.677


### Посчитаем мпетрику дообученной модели

In [34]:
# считаю метрику дообученной модели на валидационном датасете

valid_f1 = F1Score().to(device)
model.eval()

total_acc_val = 0

for val_input, val_label in valid_loader:
  
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    valid_f1(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {valid_f1.compute().item():.3f}')

Val accuracy: 0.677, F1 score: 0.677


### Дообученная модель показла метрику F1 score: 0.677, обычная модель имела метрику F1 score: 0.561